In [76]:
import torch
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 使用第二块GPU（从0开始）
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import random
import torch.nn.functional as F
import torchvision
import numpy as np
from math import *
import matplotlib.pyplot as plt
from torch.autograd import Variable
from IPython import display
import torch.utils.data as Data
import torch.nn as nn
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import cm
from scipy.linalg import block_diag
import datetime
from torch.nn.utils import *
from Transformer_model import *
import h5py

In [77]:
# 参数设置
Nc = 32       # 子载波数目
N = 2         # 路径数目
Nt = 64       # 基站侧天线数目
Nr = 1        # 用户端天线数目

L = 8         # OFDM导频数目
SNR_dB = 10   # SNR_dB
K = 2         # 用户数目
snr = 10**(SNR_dB/10)

In [78]:
def Num2Bit(Num, B):
    Num_ = Num.type(torch.uint8)

    def integer2bit(integer, num_bits=B * 2):
        dtype = integer.type()
        exponent_bits = -torch.arange(-(num_bits - 1), 1).type(dtype)
        exponent_bits = exponent_bits.repeat(integer.shape + (1,))
        out = integer.unsqueeze(-1) // 2 ** exponent_bits
        return (out - (out % 1)) % 2

    bit = integer2bit(Num_)
    bit = (bit[:, :, B:]).reshape(-1, Num_.shape[1] * B)
    return bit.type(torch.float32)


def Bit2Num(Bit, B):
    Bit_ = Bit.type(torch.float32)
    Bit_ = torch.reshape(Bit_, [-1, int(Bit_.shape[1] / B), B])
    num = torch.zeros(Bit_[:, :, 0].shape).cuda()
    for i in range(B):
        num = num + Bit_[:, :, i] * 2 ** (B - 1 - i)
    return num


class Quantization(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, B):
        ctx.constant = B
        step = 2 ** B
        out = torch.round(x * step - 0.5)
        out = Num2Bit(out, B)
        return out

    @staticmethod
    def backward(ctx, grad_output):
        # return as many input gradients as there were arguments.
        # Gradients of constant arguments to forward must be None.
        # Gradient of a number is the sum of its four bits.
        b, _ = grad_output.shape
        grad_num = torch.sum(grad_output.reshape(b, -1, ctx.constant), dim=2)
        return grad_num, None


class Dequantization(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, B):
        ctx.constant = B
        step = 2 ** B
        out = Bit2Num(x, B)
        out = (out + 0.5) / step
        return out

    @staticmethod
    def backward(ctx, grad_output):
        # return as many input gradients as there were arguments.
        # Gradients of non-Tensor arguments to forward must be None.
        # repeat the gradient of a Num for four time.
        #b, c = grad_output.shape
        #grad_bit = grad_output.repeat(1, 1, ctx.constant)
        grad_bit = grad_output.repeat_interleave(ctx.constant,dim=1)
        return grad_bit, None


class QuantizationLayer(nn.Module):

    def __init__(self, B):
        super(QuantizationLayer, self).__init__()
        self.B = B

    def forward(self, x):
        out = Quantization.apply(x, self.B)
        return out


class DequantizationLayer(nn.Module):

    def __init__(self, B):
        super(DequantizationLayer, self).__init__()
        self.B = B

    def forward(self, x):
        out = Dequantization.apply(x, self.B)
        return out

In [79]:
def Hermitian(X):#torch矩阵共轭转置
    X = torch.real(X) - 1j*torch.imag(X)
    return X.transpose(-1,-2)

class ResBlock(nn.Module):
    def __init__(self, channel_list):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=channel_list[0],  # 图片通道数  ?????
                out_channels=channel_list[1],  # filter数量
                kernel_size=(5, 1),  # filter大小
                stride=1,  # 扫描步进
                padding=(2, 0),  # 周围围上2圈0 使得输出的宽和高和之前一样不变小
            ),
            nn.BatchNorm2d(channel_list[1], eps=1e-05, momentum=0.1, affine=True),
            Mish(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=channel_list[1],  # 图片通道数
                out_channels=channel_list[2],  # filter数量
                kernel_size=(5, 1),  # filter大小
                stride=1,  # 扫描步进
                padding=(2, 0),  # 周围围上2圈0 使得输出的宽和高和之前一样不变小
            ),
            nn.BatchNorm2d(channel_list[2], eps=1e-05, momentum=0.1, affine=True),
            Mish(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels=channel_list[2],  # 图片通道数
                out_channels=channel_list[0],  # filter数量
                kernel_size=(5, 1),  # filter大小
                stride=1,  # 扫描步进
                padding=(2, 0),  # 周围围上2圈0 使得输出的宽和高和之前一样不变小
            ),
            nn.BatchNorm2d(channel_list[0], eps=1e-05, momentum=0.1, affine=True),
        )
    def forward(self, x_in):
        #residual = x_in
        x = self.conv1(x_in)
        x = self.conv2(x)
        x = self.conv3(x)
        #实例化对象
        mish = Mish()
        #数据输入，传播
        x = mish(x + x_in)
        return x

class MyLoss_OFDM(torch.nn.Module):  # 输入是信道和整个F,输出是频谱效率
    def __init__(self):
        super(MyLoss_OFDM, self).__init__()

    def forward(self, H0, out, parm_set):  # H0第0个维度是样本 第1个维度是用户，第2个维度是子载波，第3个维度是天线
        Nc = parm_set[0]
        Nt = parm_set[1]
        Nr = parm_set[2]
        snr = parm_set[3]
        B = parm_set[4]
        K = parm_set[5]

        H = H0.permute(0, 2, 1, 3)  # H第0个维度是样本 第1个维度是子载波，第2个维度是用户，第3个维度是天线
        num = out.shape[0]
        Nc = H.shape[1]
        H_real = H[:, :, :, 0:Nt]
        H_imag = H[:, :, :, Nt:2 * Nt]
        Hs = torch.zeros([num, Nc, K, Nt * 2])
        Hs = Hs.cuda()
        Hs[:, :, 0:K, 0:Nt] = H_real
        # Hs[:, :, K:2 * K, Nt:2 * Nt] = H_real
        Hs[:, :, 0:K, Nt:2 * Nt] = H_imag
        # Hs[:, :, K:2 * K, 0:Nt] = -H_imag

        F = torch.zeros([num, Nc, Nt * 2, K * 2])
        F = F.cuda()
        F[:, :, 0:Nt, 0:K] = out[:, 0:K * Nt * Nc].reshape(num, Nc, Nt, K)
        F[:, :, Nt:2 * Nt, K:2 * K] = out[:, 0:K * Nt * Nc].reshape(num, Nc, Nt, K)
        F[:, :, 0:Nt, K:2 * K] = out[:, K * Nt * Nc:2 * K * Nt * Nc].reshape(num, Nc, Nt, K)
        F[:, :, Nt:2 * Nt, 0:K] = -out[:, K * Nt * Nc:2 * K * Nt * Nc].reshape(num, Nc, Nt, K)
        R = 0
        Hk = torch.matmul(Hs, F)
        noise = 1 / snr
        for i in range(K):
            signal = Hk[:, :, i, i] * Hk[:, :, i, i] + Hk[:, :, i, i + K] * Hk[:, :, i, i + K]
            interference = torch.zeros(num, Nc)
            interference = interference.cuda()
            for j in range(K):
                if j != i:
                    interference = interference + Hk[:, :, i, j] * Hk[:, :, i, j] + Hk[:, :, i, j + K] * Hk[:, :, i,
                                                                                                         j + K]
            SINR = signal / (noise + interference)
            R = R + torch.sum(torch.log2(1 + SINR))
        R = -R / num / Nc
        return R
    
class DatasetFolder(Data.Dataset):
    def __init__(self, matData):
        self.matdata = matData
    def __getitem__(self, index):
        return self.matdata[index]
    def __len__(self):
        return self.matdata.shape[0]

class NoamOpt(object):
    "Optim wrapper that implements rate."

    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0

    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()

    def rate(self, step=None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
             min(step ** (-0.5), step * self.warmup ** (-1.5)))


def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
                   torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
def mish(x):
    x = x * (torch.tanh(F.softplus(x)))
    return x


class Mish(nn.Module):
    def __init__(self):
        super().__init__()

    #         print("Mish activation loaded...")
    def forward(self, x):
        x = x * (torch.tanh(F.softplus(x)))
        return x
    
def DFT_matrix(N):
    i, j = np.meshgrid(np.arange(N), np.arange(N))
    omega = np.exp(- 2 * pi * 1J / N)
    W = np.power(omega, i * j) / sqrt(N)
    return np.mat(W)

Nc = 32
Nt = 64
W = DFT_matrix(Nc)
W = torch.tensor(W)
W = Hermitian(W)
W_real = np.real(W).cuda()
W_imag = np.imag(W).cuda()
W_real = W_real.float()
W_imag = W_imag.float()

W1 = DFT_matrix(Nt)
W_real1 = torch.from_numpy(np.real(W1)).cuda()
W_imag1 = torch.from_numpy(np.imag(W1)).cuda()
W_real1 = W_real1.float()
W_imag1 = W_imag1.float()

RuntimeError: CUDA error: the launch timed out and was terminated
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
class DNN_US_RF_OFDM(nn.Module):
    def __init__(self, pram_set):
        Nc = pram_set[0]  #子载波数目
        Nt = pram_set[1]  #发送天线数目
        Nr = pram_set[2]  #接收天线数目
        snr = pram_set[3] #信噪比
        B = pram_set[4]   #量化的比特数目
        K = pram_set[5]   #用户数目
        super(DNN_US_RF_OFDM, self).__init__()

        self.pilot = nn.Linear(Nt, L, bias=False)  # 全连接

        self.res = ResBlock([2 * L, 256, 512])

        self.FC2 = nn.Linear(2 * Nc * L, 1024)  # 全连接
        self.bn2 = nn.BatchNorm1d(1024)
        self.mish2 = Mish()

        self.FC3 = nn.Linear(1024, 512)  # 全连接
        self.bn3 = nn.BatchNorm1d(512)
        self.mish3 = Mish()

        self.FC4 = nn.Linear(512, B)
        self.bn4 = nn.BatchNorm1d(B)

        self.QL = QuantizationLayer(1)
    def forward(self, h, pram_set):
        Nc = pram_set[0]
        Nt = pram_set[1]
        Nr = pram_set[2]
        snr = pram_set[3]
        B = pram_set[4]
        K = pram_set[5]

        # Fig3 HPN网络
        h_real = h[:, :, 0:Nt].reshape(-1, Nc, Nt, 1)
        h_imag = h[:, :, Nt:2 * Nt].reshape(-1, Nc, Nt, 1)

        F_real = torch.cos(self.pilot.weight) / sqrt(Nt) * sqrt(K)  #改成NC子载波，一共有两处要更改,还是除以K
        F_imag = torch.sin(self.pilot.weight) / sqrt(Nt) * sqrt(K)

        L_real = (torch.matmul(F_real, h_real) - torch.matmul(F_imag, h_imag)).reshape(-1, 1, Nc, L)
        L_imag = (torch.matmul(F_real, h_imag) + torch.matmul(F_imag, h_real)).reshape(-1, 1, Nc, L)

        L_sum = torch.cat((L_real, L_imag), 1)
        # shape[0]输出矩阵行数
        # shape[1]输出矩阵列数
        num = h.shape[0]
        noise = torch.randn(num, 2, Nc, L) / sqrt(2 * snr)
        noise = noise.cuda()
        L_sum = L_sum + noise   #由H得到Y
        # DFT运算
        L_real = (torch.matmul(W_real, L_sum[:, 0, :, :]) - torch.matmul(W_imag, L_sum[:, 1, :, :])).reshape(-1, Nc, L)
        L_imag = (torch.matmul(W_imag, L_sum[:, 0, :, :]) + torch.matmul(W_real, L_sum[:, 1, :, :])).reshape(-1, Nc, L)
        L_sum = torch.cat((L_real, L_imag), 2)

        x = L_sum.transpose(1, 2)
        x = x.reshape(-1, 2 * L, Nc, 1)
        x = self.res(x)
        x = x.reshape(-1, 2 * L * Nc)    # reshape操作，真值表示

        x = self.FC2(x)
        x = self.bn2(x)
        x = self.mish2(x)

        x = self.FC3(x)
        x = self.bn3(x)
        x = self.mish3(x)

        x = self.FC4(x)
        x = self.bn4(x)

        x = torch.sigmoid(x)
        x = self.QL(x)
        return x

In [ ]:
class RIS_SDMA_Precoding(nn.Module): #单层
    def __init__(self, parm_set): 
        super(RIS_SDMA_Precoding,self).__init__()
        
        Nc = parm_set[0]
        Nt = parm_set[1]
        Nr = parm_set[2]
        snr = parm_set[3]
        B = parm_set[4]
        K = parm_set[5]

        self.DQL = DequantizationLayer(1)  # 解1bit 量化

        self.FC1 = nn.Linear(K * B, Nc*K*Nt*2)  # 全连接
        self.bn1 = nn.BatchNorm1d(Nc*K*Nt*2)
        self.mish1 = Mish()
        
        
        self.linear_H  = nn.Linear(Nt*2, 32)
        
        self.trans1  = TRANS_BLOCK(32*K,32,256,3)
        self.linear_RIS = nn.Linear(Nc*32, Nt)   #生成RIS相位

        self.linear_RF = nn.Linear(Nc*32, Nt*K)   #生成RIS相位
        
        self.trans2  = TRANS_BLOCK(2*K*K,4*K,256,3) #4×4等效信道
        
    
    def forward(self, out,parm_set):
        # H(batch,K,Nc,Nt)
        #H_RU [batch,K,Nc,1,M_ant]  s [batch,Nc,2*K] H_BR[1,Nc,M_ant,M_ant]
        # param_list = [fc,B,Nc,M,N,D_sub,D_ant,R,M_ant,h1,h2,L,SNR]
        Nc = parm_set[0]
        Nt = parm_set[1]
        Nr = parm_set[2]
        snr = parm_set[3]
        B = parm_set[4]
        K = parm_set[5]
        

        x = self.DQL(out) - 0.5

        x = x.contiguous().view(-1,int(K*B))
        x = self.FC1(x)
        x = self.bn1(x)
        x = self.mish1(x)

        H = x.contiguous().view(-1,int(K),int(Nc),int(2*Nt))   #不完美信道
        batch = H.shape[0]


        '''
        #H [batch,K,Nc,2*Nt]
        x = self.linear_H(H)   #[batch,K,Nc,32]
        x = x.permute(0,2,1,3) #[batch,Nc,K,32]
        x = x.reshape(-1,Nc,K*32)#[batch,Nc,K*32]
        
        out_RF = x[:,:,32:64].reshape(-1,Nc*32)
        RF_phase = self.linear_RF(out_RF)
        F_RF = torch.exp(1j*RF_phase.reshape(-1,Nt,K))/sqrt(Nt) #[batch,1,M_ant,K]
        '''
        #H[batch,K,Nc,Nt]
        H1 = torch.complex(H[:,:,:,0:Nt],H[:,:,:,:Nt:2*Nt])     #合并完成为复数矩阵
        
        F_RF = (torch.zeros(batch,Nt,K)+ 0j).cuda()
        #不同用户的模拟预编码矩阵不同
        for i in range(K):
            F_RF[:,:,i] = H1[:,i,Nc//2,:]
        F_RF = torch.real(F_RF) - 1j*torch.imag(F_RF)        #共轭转置
        F_RF = (F_RF/torch.abs(F_RF))/sqrt(Nt)
        
        #print(H1.size())
        #print(F_RF.size())
        H_equ = (torch.zeros(batch,Nc,K,K) + 0j).cuda()
        H1 = H1.reshape([Nc,batch,K,Nt])
        H_equ = (H1 @ F_RF).reshape([batch,Nc,K,K])    #(batch,K,Nc,K)
           
        n = (torch.randn(H.shape[0],Nc,K) + 1j*torch.randn(H.shape[0],Nc,K)).cuda()/sqrt(2 * snr)   #噪声
        
        power = torch.sum(torch.abs(H_equ*H_equ),[2,3])
        H_equ = H_equ/torch.sqrt(power.reshape(-1,Nc,1,1))
        

        H_equ = H_equ.reshape(-1,Nc,K*K)
        x = torch.cat((torch.real(H_equ),torch.imag(H_equ)), 2) #[batch,Nc,2*K*K] 实数
        x = self.trans2(x)   #[batch,Nc,4*K+1]

        H_equ = H_equ.reshape(-1,Nc,K,K)

        H_hat = H_equ.to(torch.complex128)
        U = (x[:,:,0:K] + 1j*x[:,:,K:2*K]).to(torch.complex128)    #[batch,Nc,K]
        W = (x[:,:,2*K:3*K] + 1j*x[:,:,3*K:4*K]).to(torch.complex128)   #[batch,Nc,K] 相当于是U、W的更新函数
        # pub1 = (x[:,:,4*K:5*K] + 1j*x[:,:,5*K:6*K]).to(torch.complex128) #[batch,Nc,K]
        # pub2 = (x[:,:,6*K:7*K] + 1j*x[:,:,7*K:8*K]).to(torch.complex128) #[batch,Nc,K]

        #sigma_pri = (x[:,:,4*K]).to(torch.complex128)  #[batch,Nc]
        # sigma_pub = (x[:,:,8*K+1]).to(torch.complex128)#[batch,Nc]

        # 现在来更新V
        max_iter = 100
        iter1=0
        min_mu = 0
        max_mu = 10
        P = 1  #发射功率
        while 1:
            mu = (min_mu + max_mu) / 2
            
            for k in range(K):
                hk = Hermitian(H_hat[:,:,k,:].reshape(batch,Nc,1,K))
                if(k==0):
                    B_pri = W[:,:,k].reshape(batch,Nc,1,1) * torch.abs(U[:,:,k].reshape(batch,Nc,1,1))**2 * hk @ Hermitian(hk)
                    B_pri = B_pri + (mu *torch.eye(K).cuda().to(torch.complex128))
                # B_pub = pub2[:,:,k].reshape(batch,Nc,1,1) * hk @ Hermitian(hk)
                # B_pub = B_pub + (sigma_pub.reshape(batch,Nc,1,1) * torch.eye(K).cuda().to(torch.complex128))
                else:
                #二分法算mu

                    B_pri = B_pri + W[:,:,k].reshape(batch,Nc,1,1) * torch.abs(U[:,:,k].reshape(batch,Nc,1,1))**2 * hk @ Hermitian(hk)
                #B_pri = B_pri + pri2[:,:,k].reshape(batch,Nc,1,1) * hk @ Hermitian(hk)
                #B_pri = B_pri + (sigma_pri.reshape(batch,Nc,1,1) * torch.eye(K).cuda().to(torch.complex128))
                    B_pri = B_pri + (mu *torch.eye(K).cuda().to(torch.complex128))
                # B_pub = B_pub + pub2[:,:,k].reshape(batch,Nc,1,1) * hk @ Hermitian(hk)
                # B_pub = B_pub + (sigma_pub.reshape(batch,Nc,1,1) * torch.eye(K).cuda().to(torch.complex128))
        
            V_pri = (torch.zeros((batch,Nc,K,K)) + 0j).cuda().to(torch.complex128)
        # V_pub = (torch.zeros((batch,Nc,K,1)) + 0j).cuda().to(torch.complex128)
        # B_inv_pub = torch.inverse(B_pub)
            B_inv_pri = torch.inverse(B_pri)
        # print(np.sum(np.abs(A_inv)**2))
            for k in range(K):
                hk = Hermitian(H_hat[:,:,k,:].reshape(batch,Nc,1,K))
            #V_pri[:,:,:,k] = (B_inv_pri @ (pri1[:,:,k].reshape(batch,Nc,1,1) * hk)).reshape(batch,Nc,K)
                V_pri[:,:,:,k] = ((U[:,:,k].reshape(batch,Nc,1,1) * W[:,:,k].reshape(batch,Nc,1,1) * B_inv_pri) @ hk).reshape(batch,Nc,K)
        # for k in range(K):
        #     hk = Hermitian(H_hat[:,:,k,:].reshape(batch,Nc,1,K))
        #     if k==0:
        #         A = pub1[:,:,k].reshape(batch,Nc,1,1) * hk
        #     else:
        #         A = A + pub1[:,:,k].reshape(batch,Nc,1,1) * hk
        # V_pub = (B_inv_pub @ A)
            Pitem = torch.sum(torch.abs(V_pri*V_pri))  #
            if Pitem> P:   #功率过大 mu在分母上
                min_mu = mu
            else:
                max_mu = mu
            iter1 = iter1 + 1
            if ((max_mu - min_mu) < 1e-5 or iter1 > max_iter):
                break
            #找到mu
        for k in range(K):
            hk = Hermitian(H_hat[:,:,k,:].reshape(batch,Nc,1,K))
            if(k==0):
                B_pri = W[:,:,k].reshape(batch,Nc,1,1) * torch.abs(U[:,:,k].reshape(batch,Nc,1,1))**2 * hk @ Hermitian(hk)
                B_pri = B_pri + (mu *torch.eye(K).cuda().to(torch.complex128))
                # B_pub = pub2[:,:,k].reshape(batch,Nc,1,1) * hk @ Hermitian(hk)
                # B_pub = B_pub + (sigma_pub.reshape(batch,Nc,1,1) * torch.eye(K).cuda().to(torch.complex128))
            else:
                #二分法算mu

                B_pri = B_pri + W[:,:,k].reshape(batch,Nc,1,1) * torch.abs(U[:,:,k].reshape(batch,Nc,1,1))**2 * hk @ Hermitian(hk)
                #B_pri = B_pri + pri2[:,:,k].reshape(batch,Nc,1,1) * hk @ Hermitian(hk)
                #B_pri = B_pri + (sigma_pri.reshape(batch,Nc,1,1) * torch.eye(K).cuda().to(torch.complex128))
                B_pri = B_pri + (mu *torch.eye(K).cuda().to(torch.complex128))
                # B_pub = B_pub + pub2[:,:,k].reshape(batch,Nc,1,1) * hk @ Hermitian(hk)
                # B_pub = B_pub + (sigma_pub.reshape(batch,Nc,1,1) * torch.eye(K).cuda().to(torch.complex128))
        
        V_pri = (torch.zeros((batch,Nc,K,K)) + 0j).cuda().to(torch.complex128)
        # V_pub = (torch.zeros((batch,Nc,K,1)) + 0j).cuda().to(torch.complex128)
        # B_inv_pub = torch.inverse(B_pub)
        B_inv_pri = torch.inverse(B_pri)
        # print(np.sum(np.abs(A_inv)**2))
        for k in range(K):
            hk = Hermitian(H_hat[:,:,k,:].reshape(batch,Nc,1,K))
            #V_pri[:,:,:,k] = (B_inv_pri @ (pri1[:,:,k].reshape(batch,Nc,1,1) * hk)).reshape(batch,Nc,K)
            V_pri[:,:,:,k] = ((U[:,:,k].reshape(batch,Nc,1,1) * W[:,:,k].reshape(batch,Nc,1,1) * B_inv_pri) @ hk).reshape(batch,Nc,K)

        w_pub = (torch.zeros((batch,Nc,K,1)) + 0j).cuda().to(torch.complex128)  #没有public signal
        W_pri = V_pri


        F_BB_RS = (w_pub + 0).to(torch.complex64)

        F_BB_SDMA = (W_pri + 0).to(torch.complex64)   #[batch,Nc,K,K]
        F_BB_SDMA = F_BB_SDMA.reshape([Nc,batch,K,K]) # digital precoding
        F_SDMA = (F_RF @ F_BB_SDMA).to(torch.complex64)  #[batch,Nc,M_ant,K]         [batch,Nt,K] * [Nc,batch,K,K] = [Nc,batch,Nt,K]广播机制
        #F_RS = F_RF @ F_BB_RS  #[batch,Nc,M_ant,N_RS1]
        F_SDMA = F_SDMA.reshape([batch,Nc,Nt,K])
        Power = (torch.sum(torch.abs(F_SDMA)**2,[2,3])).reshape(-1,Nc,1,1)   #[batch,Nc,1,1]  [batch,Nc,Nt,K]
        #Power1 = (torch.sum(torch.abs(F_SDMA)**2,[1,2,3])).reshape(-1,1,1,1)
        #print(Power1.size())
        #print(F_BB_SDMA.size())
        F_BB_SDMA = F_BB_SDMA.permute(1,0,2,3)
        #F_SDMA = F_SDMA / torch.sqrt(Power) * sqrt(K)
        F_BB_SDMA = F_BB_SDMA / torch.sqrt(Power)  #F_BB做归一化
        #F_BB_RS = F_BB_RS / torch.sqrt(Power) * sqrt(K)
        #归一化之后的预编码矩阵
        
        F_BB_SDMA = F_BB_SDMA.permute(1,0,2,3)
        F_SDMA = F_RF @ F_BB_SDMA
        F_SDMA = F_SDMA.permute(1,0,2,3)
        F_SDMA = F_SDMA.reshape([batch,K*Nc*Nt])

    
        #abs(A)**2
        F_real = torch.real(F_SDMA).reshape(batch, K * Nt * Nc)
        F_imag = torch.imag(F_SDMA).reshape(batch, K * Nt * Nc)
        F = torch.cat((F_real, F_imag), 1)
        return F  #Phi[batch,1,M_ant,M_ant]  F_RF[batch,1,M_ant,K]  F_BB_SDMA[batch,Nc,K,K] F_BB_RS[batch,Nc,K,1] 
#         print(H_RU.shape)

In [ ]:
def train(Nc, N, Nt, B, Nr, L, SNR_dB, K, EPOCH, BATCH_SIZE):  # N代表路径数，若N为-1则代表
    shoulian = np.zeros(EPOCH)
    snr = 10 ** (SNR_dB / 10)
    parm_set = [Nc, Nt, Nr, snr, B, K]

    # H_train = torch.load('data/H_train_UPA' + str(N) + 'Lp.pt')
    # H_train = H_train[:, 0:K, :, :]
    # print(H_train.shape)
    # H_test = torch.load('data/H_test_UPA' + str(N) + 'Lp.pt')
    # H_test = H_test[:, 0:K, :, :]
    # print(H_test.shape)
    train = 'H_train_'+str(N)+'.mat'
    mat = h5py.File(train)
    H_train = mat['H_train']
    H_train = np.transpose(H_train, [3, 2, 1, 0])
    H_train = H_train.astype('float32')  # 训练变量类型转换
    print(H_train.shape)
    test = 'H_val_'+str(N)+'.mat'
    mat = h5py.File(test)
    H_test = mat['H_val']
    H_test = np.transpose(H_test, [3, 2, 1, 0])
    H_test = H_test.astype('float32')  # 训练变量类型转换

    net_US = DNN_US_RF_OFDM(parm_set).cuda()
    net_BS = RIS_SDMA_Precoding(parm_set).cuda()

    optimizer_US = torch.optim.Adam(net_US.parameters(), lr=0.001)
    scheduler_US = torch.optim.lr_scheduler.MultiStepLR(optimizer_US, milestones=[100, 150], gamma=0.6,
                                                        last_epoch=-1)  # 训练过程中动态调整学习率
    
    opt = NoamOpt(256, 1, 4000, torch.optim.Adam(net_BS.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
    #optimizer_BS = torch.optim.Adam(net_BS.parameters(), lr=0.001)
    #scheduler_BS = torch.optim.lr_scheduler.MultiStepLR(optimizer_BS, milestones=[100, 150], gamma=0.6, last_epoch=-1)

    loss_func1 = MyLoss_OFDM()
    loss_func1 = loss_func1.cuda()

    best_SE = 0
    torch_dataset_train = DatasetFolder(H_train)
    loader_train = Data.DataLoader(
        dataset=torch_dataset_train,
        batch_size=BATCH_SIZE,
        shuffle=True,  # 训练时随机打乱数据
        num_workers=0,  # 每次用两个进程提取
    )

    torch_dataset_test = DatasetFolder(H_test)
    loader_test = Data.DataLoader(
        dataset=torch_dataset_test,
        batch_size=BATCH_SIZE,
        shuffle=True,  # 训练时随机打乱数据
        num_workers=0,  # 每次用两个进程提取
    )
    start = datetime.datetime.now()
    for epoch in range(EPOCH):
        print('========================')
        print('lr:%.4e' % opt.optimizer.param_groups[0]['lr'])
        train_SE = 0
        num_train = 0
        test_SE = 0
        num_test = 0
        for step, b_x in enumerate(loader_train):
            num_train = num_train + 1
            net_BS.train()  # 训练模式
            net_US.train()
            b_x = b_x.cuda()
            num = b_x.shape[0]
            out1 = torch.zeros([num, B*K]) # 用户端K个用户量化比特输出，K个用户共用一个网络
            out1 = out1.cuda()
            #print(out1.dtype)
            for i in range(K):
                out1k = net_US(b_x[:,i,:,:],parm_set)   #信道是四维矩阵，pram_set为五维矩阵,PFN网络，生成B比特数据
                #print(net_US(b_x[:,i,:,:],parm_set))
                out1[:, i*B : (i * B + B)] = out1k
            out2 = net_BS(out1, parm_set)
            loss = loss_func1(b_x, out2, parm_set)
            train_SE = train_SE - loss
            optimizer_US.zero_grad()
            opt.optimizer.zero_grad()

            loss.backward()
            optimizer_US.step()
            opt.step()
        train_SE = train_SE / num_train
        scheduler_US.step()
        # scheduler_BS.step()
        net_BS.eval()
        net_US.eval()
        with torch.no_grad():
            for step, b_x in enumerate(loader_test):
                num_test = num_test + 1
                net_BS.eval()  # 验证模式
                net_US.eval()
                b_x = b_x.cuda()
                num = b_x.shape[0]
                out1 = torch.zeros([num, B*K]) # 用户端K个用户量化比特输出，K个用户共用一个网络
                out1 = out1.cuda()
            #print(out1.dtype)
                for i in range(K):
                    out1k = net_US(b_x[:,i,:,:],parm_set)   #信道是四维矩阵，pram_set为五维矩阵,PFN网络，生成B比特数据
                #print(net_US(b_x[:,i,:,:],parm_set))
                    out1[:, i*B : (i * B + B)] = out1k
                out2 = net_BS(out1, parm_set)
                loss = loss_func1(b_x, out2, parm_set)
                test_SE = test_SE - loss
            test_SE = test_SE / num_test

        time0 = datetime.datetime.now() - start
        print('Epoch:', epoch, 'time', time0, 'train SE %.3f' % train_SE.cpu(), 'test SE %.3f' % test_SE.cpu())
        start = datetime.datetime.now()

        if test_SE > best_SE:
            best_SE = test_SE
            torch.save(net_US,
                       './US_WMMSE_diff_' + str(B) + 'B' + str(N) + 'Ncl' + str(L) + 'L' + str(K) + 'K' + '_8L_UPA_T01_Trans_warm.pth')
            torch.save(net_BS,
                       './BS_WMMSE_diff_' + str(B) + 'B' + str(N) + 'Ncl' + str(L) + 'L' + str(K) + 'K' + '_8L_UPA_T01_Trans_warm.pth')
            print('Model saved!')
        shoulian[epoch] = test_SE.cpu()
    print('The best SE is: %.3f' % best_SE.cpu())
    print(shoulian)

Nc = 32
# N = 2  # 多径数
Nt = 64 # 基站端天线数
Nr = 1 # 用户端天线数
# B = 30

L = 8  # 用户端接收的观测，即OFDM个数
SNR_dB = 10
K = 2  # 用户数
snr = 10**(SNR_dB/10)

BATCH_SIZE = 512
EPOCH = 180

#B = 40  # 反馈bit数
#N_=[5,6]
Bs = [10,20,30,40,50,60,70]
N=6
if __name__ == '__main__':
    for B in Bs:
        train(Nc,N,Nt,B,Nr,L,SNR_dB,K,EPOCH,BATCH_SIZE)

(100000, 2, 32, 128)
lr:0.0000e+00


C:\Users\y\AppData\Local\Temp\ipykernel_19460\230855352.py:8: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  out = integer.unsqueeze(-1) // 2 ** exponent_bits


Epoch: 0 time 0:02:10.344344 train SE 3.326 test SE 6.235
Model saved!
lr:4.8422e-05
Epoch: 1 time 0:02:07.559301 train SE 5.963 test SE 9.141
Model saved!
lr:9.6845e-05
Epoch: 2 time 0:02:05.470381 train SE 7.470 test SE 10.668
Model saved!
lr:1.4527e-04
Epoch: 3 time 0:02:06.925993 train SE 8.686 test SE 10.971
Model saved!
lr:1.9369e-04
Epoch: 4 time 0:02:06.821785 train SE 9.743 test SE 11.776
Model saved!
lr:2.4211e-04
Epoch: 5 time 0:02:06.445280 train SE 10.714 test SE 12.359
Model saved!
lr:2.9053e-04
Epoch: 6 time 0:02:07.541349 train SE 11.555 test SE 12.733
Model saved!
lr:3.3896e-04
Epoch: 7 time 0:02:06.248301 train SE 12.005 test SE 12.433
lr:3.8738e-04
Epoch: 8 time 0:02:06.362994 train SE 12.100 test SE 11.824
lr:4.3580e-04
Epoch: 9 time 0:02:06.290694 train SE 12.441 test SE 13.248
Model saved!
lr:4.8422e-04
Epoch: 10 time 0:02:05.861373 train SE 13.089 test SE 12.880
lr:5.3265e-04
Epoch: 11 time 0:02:06.306652 train SE 13.297 test SE 14.097
Model saved!
lr:5.8107e-04


RuntimeError: CUDA error: the launch timed out and was terminated
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.